# Downloading stock data

In [65]:
from datetime import datetime
import pandas as pd
from openbb import obb
import os

In [66]:

obb.user.credentials.fmp_api_key="edc2977d918bfc0e71920ca88e986b39"
data_folder = 'data'
company = 'SPY'
interval = '1d' # 1m, 1h, 1d options
start_date = '2004-01-01'
fn = f"{company}_{start_date}_{interval}.csv"
out_path = os.path.join(data_folder, company)
df = obb.equity.price.historical(company, interval, start_date, provider="fmp").to_df()
df

,open,high,low,close,volume,vwap,label,adj_close,unadjusted_volume,change,change_percent,change_over_time
date,,,,,,,,,,,,
2004-01-02,111.740,112.1900,110.73,111.23,38072300,111.38,"January 02, 04",75.87,38072300.0,-0.51,-0.456420,-0.004564
2004-01-05,111.690,112.5200,111.59,112.44,27959800,112.18,"January 05, 04",76.69,27959800.0,0.75,0.671500,0.006715
2004-01-06,112.160,112.7300,112.00,112.55,20472800,112.43,"January 06, 04",76.77,20472800.0,0.39,0.347720,0.003477
2004-01-07,112.390,113.0600,111.89,112.93,30170400,112.63,"January 07, 04",77.03,30170400.0,0.54,0.480470,0.004805
2004-01-08,113.250,113.4100,112.77,113.38,36438400,113.19,"January 08, 04",77.33,36438400.0,0.13,0.114790,0.001148
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-30,490.560,491.6200,490.11,490.89,57658386,490.87,"January 30, 24",490.89,58618400.0,0.33,0.067270,0.000673
2024-01-31,488.620,489.0813,482.86,482.88,126011061,485.89,"January 31, 24",482.88,126011100.0,-5.74,-1.170000,-0.011700
2024-02-01,484.630,489.2300,483.80,489.20,91891637,486.58,"February 01, 24",489.20,91891600.0,4.57,0.942990,0.009430


In [63]:
df.to_csv(os.path.join(out_path, fn))

In [102]:
obb.user.credentials.fmp_api_key="edc2977d918bfc0e71920ca88e986b39"
data_folder = 'data'
company = 'NDAQ'
interval = '1h' # 1m, 1h, 1d options
start_date = '2004-01-01'
fn = f"{company}_{start_date}_{interval}.csv"
out_path = os.path.join(data_folder, company)
df = obb.equity.price.historical(company, interval, start_date, provider="yfinance").to_df()
df

,open,high,low,close,volume,dividends,stock_splits
date,,,,,,,
2022-02-07 09:30:00,60.22,60.33,59.54,59.97,200183,0.0,0.0
2022-02-07 10:30:00,60.01,60.19,59.82,59.92,84996,0.0,0.0
2022-02-07 11:30:00,59.94,59.95,59.64,59.68,58616,0.0,0.0
2022-02-07 12:30:00,59.67,59.88,59.64,59.85,56593,0.0,0.0
2022-02-07 13:30:00,59.86,59.92,59.64,59.82,59600,0.0,0.0
...,...,...,...,...,...,...,...
2024-02-05 11:30:00,55.92,55.97,55.78,55.96,241066,0.0,0.0
2024-02-05 12:30:00,55.97,56.18,55.96,56.02,100593,0.0,0.0
2024-02-05 13:30:00,56.03,56.09,55.99,56.07,106984,0.0,0.0


In [97]:
print(df.idxmin()[0])

2022-10-13 09:30:00


In [110]:
# for saving hourly data for each company
folders = os.listdir(data_folder)
if '.DS_Store' in folders:
    folders.pop(folders.index('.DS_Store'))
interval = '1h'
start_date = '2004-01-01'
for company in folders:
    already_downloaded = False
    files = os.listdir(os.path.join(data_folder, company))

    if len(files) != 0:
        for fn in files:
            if interval in fn:
                already_downloaded = True
                print(f"Skipping {company}, data already downloaded!")

    if not already_downloaded:
        print(f"Looking for {company}...")
        df = obb.equity.price.historical(company, interval, start_date, provider="yfinance").to_df()
        min_date = df.idxmin()[0]
        date_str = f'{min_date.year}-{min_date.month}-{min_date.day}'
        fn = f"{company}_{date_str}_{interval}.csv"
        out_file = os.path.join(data_folder, company, fn)
        df.to_csv(out_file)
        print(f"Data saved to {out_file}!\n")

Skipping AMZN, data already downloaded!
Skipping SPY, data already downloaded!
Skipping AAPL, data already downloaded!
Skipping GOOG, data already downloaded!
Skipping MSFT, data already downloaded!
Skipping META, data already downloaded!
Skipping NFLX, data already downloaded!
Skipping NDAQ, data already downloaded!
Skipping NVDA, data already downloaded!
Looking for AVGO...
Data saved to data/AVGO/AVGO_2022-10-13_1h.csv!

